# Classes RelationalQueryProcessor and TriplestoreQueryProcessor
Add the following method to both the classes:

## searchByAuthor(name : str) : DataFrame
It returns a data frame with all the publications (i.e. the rows) that have been authored by people having the input string included either in their family name or given name. The match must be done lowercase and can be also partial, e.g. the input string “jo" will match with “Jo", “John", “Johnathan" and “Guido".

# Class GenericQueryProcessor
Add the following method to the class:

## searchByAuthor(name : str) : list[Publication]
It returns a list of Publication objects referring to all the publications (i.e. the rows) that have been authored by people having the input string included either in their family name or given name. The match must be done lowercase and can be also partial, e.g. the input string “jo" will match with “Jo", “John", “Johnathan" and “Guido".


# Implementation
I will define my data model using Python classes

In [ ]:

class Publication(object):
    def __init__(self, id, title, type, publication_year, issue, volume, chapter, publication_venue, venue_type, publisher):
        self.id = id
        self.title = title     
        self.type = type      
        self.publicationYear = publication_year
        self.issue = issue
        self.volume = volume
        self.chapter = chapter        
        self.publicationVenue = publication_venue
        self.venueType = venue_type  
        self.publisher = publisher 

#in the relational_other_data.csv we have all 500 the publication items. 
# The header contains:
id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event

# in the relational_other_data.json we have 4 dictionaries:
 1. authors: identified by family, given (names) and orcid
 2. venues_id: identified by doi and related issn (6710)
 3. references: doi in doi (8449)
 4. publisher: identified by id and name in a crossref value (9286)

In [3]:
import pandas as pd
from pandas import read_csv
from pandas import read_json
from pandas import read_sql
from pandas import merge
from pandas import concat
from pandas import Series
import sqlite3
import json
from sqlite3 import connect
from csv import reader

with open("import/relational_publications.csv", "r", encoding="utf-8") as f:
    publications = reader(f)

#loading the json file
from json import load

with open("import/relational_other_data.json", "r", encoding="utf-8") as f:
    json_doc = load(f)

print(type(json_doc))



from pandas import read_csv

df_publications = read_csv("import/relational_publications.csv")
print(df_publications)  


df_publications

<class 'dict'>
                                    id  \
0              doi:10.1162/qss_a_00023   
1       doi:10.1007/s11192-019-03217-6   
2       doi:10.1007/s11192-019-03311-9   
3            doi:10.1038/sdata.2016.18   
4     doi:10.1371/journal.pbio.3000385   
..                                 ...   
495         doi:10.3390/admsci10030069   
496     doi:10.1186/s12888-020-02825-4   
497  doi:10.1080/00472778.2020.1776578   
498     doi:10.1007/s11301-020-00196-4   
499  doi:10.1080/08989621.2020.1836620   

                                                 title             type  \
0    Opencitations, An Infrastructure Organization ...  journal-article   
1    Software Review: Coci, The Opencitations Index...  journal-article   
2    Nine Million Book Items And Eleven Million Cit...  journal-article   
3    The Fair Guiding Principles For Scientific Dat...  journal-article   
4    The Nih Open Citation Collection: A Public Acc...  journal-article   
..                            

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
0,doi:10.1162/qss_a_00023,"Opencitations, An Infrastructure Organization ...",journal-article,2020,1,1,NaN,Quantitative Science Studies,journal,crossref:281,NaN
1,doi:10.1007/s11192-019-03217-6,"Software Review: Coci, The Opencitations Index...",journal-article,2019,2,121,NaN,Scientometrics,journal,crossref:297,NaN
2,doi:10.1007/s11192-019-03311-9,Nine Million Book Items And Eleven Million Cit...,journal-article,2019,2,122,NaN,Scientometrics,journal,crossref:297,NaN
3,doi:10.1038/sdata.2016.18,The Fair Guiding Principles For Scientific Dat...,journal-article,2016,1,3,NaN,Scientific Data,journal,crossref:297,NaN
4,doi:10.1371/journal.pbio.3000385,The Nih Open Citation Collection: A Public Acc...,journal-article,2019,10,17,NaN,Plos Biology,journal,crossref:340,NaN
...,...,...,...,...,...,...,...,...,...,...,...
495,doi:10.3390/admsci10030069,Performance Analysis And Science Mapping Of In...,journal-article,2020,3,10,NaN,Administrative Sciences,journal,crossref:1968,NaN
496,doi:10.1186/s12888-020-02825-4,Mapping The Literature On Parents With Mental ...,journal-article,2020,1,20,NaN,Bmc Psychiatry,journal,crossref:297,NaN
497,doi:10.1080/00472778.2020.1776578,Evolution Of The Entrepreneurship And Innovati...,journal-article,2020,NaN,NaN,NaN,Journal Of Small Business Management,journal,crossref:301,NaN
498,doi:10.1007/s11301-020-00196-4,Intellectual Structure Of Management Innovatio...,journal-article,2020,3,71,NaN,Management Review Quarterly,journal,crossref:297,NaN


In [4]:
df_publications = read_csv("import/relational_publications.csv", 
                           keep_default_na=False,
                           dtype={
                               "id": "string",
                               "title": "string",
                               "type": "string",
                               "publication year": "int",
                               "issue": "string",
                               "volume": "string",
                               "chapter": "string",
                               "publication venue": "string",
                               "venueType": "string",
                               "publisher": "string"        
       
                           })
df_publications

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
0,doi:10.1162/qss_a_00023,"Opencitations, An Infrastructure Organization ...",journal-article,2020,1,1,,Quantitative Science Studies,journal,crossref:281,
1,doi:10.1007/s11192-019-03217-6,"Software Review: Coci, The Opencitations Index...",journal-article,2019,2,121,,Scientometrics,journal,crossref:297,
2,doi:10.1007/s11192-019-03311-9,Nine Million Book Items And Eleven Million Cit...,journal-article,2019,2,122,,Scientometrics,journal,crossref:297,
3,doi:10.1038/sdata.2016.18,The Fair Guiding Principles For Scientific Dat...,journal-article,2016,1,3,,Scientific Data,journal,crossref:297,
4,doi:10.1371/journal.pbio.3000385,The Nih Open Citation Collection: A Public Acc...,journal-article,2019,10,17,,Plos Biology,journal,crossref:340,
...,...,...,...,...,...,...,...,...,...,...,...
495,doi:10.3390/admsci10030069,Performance Analysis And Science Mapping Of In...,journal-article,2020,3,10,,Administrative Sciences,journal,crossref:1968,
496,doi:10.1186/s12888-020-02825-4,Mapping The Literature On Parents With Mental ...,journal-article,2020,1,20,,Bmc Psychiatry,journal,crossref:297,
497,doi:10.1080/00472778.2020.1776578,Evolution Of The Entrepreneurship And Innovati...,journal-article,2020,,,,Journal Of Small Business Management,journal,crossref:301,
498,doi:10.1007/s11301-020-00196-4,Intellectual Structure Of Management Innovatio...,journal-article,2020,3,71,,Management Review Quarterly,journal,crossref:297,


In [5]:
for idx, row in df_publications.iterrows():
    print("\nThe index of the current row is", idx)
    print("The content of the row is as follows:")
    print(row)


The index of the current row is 0
The content of the row is as follows:
id                                             doi:10.1162/qss_a_00023
title                Opencitations, An Infrastructure Organization ...
type                                                   journal-article
publication_year                                                  2020
issue                                                                1
volume                                                               1
chapter                                                               
publication_venue                         Quantitative Science Studies
venue_type                                                     journal
publisher                                                 crossref:281
event                                                                 
Name: 0, dtype: object

The index of the current row is 1
The content of the row is as follows:
id                                      doi:10.100

The index of the current row is 281
The content of the row is as follows:
id                                               doi:10.1111/rec.12899
title                Evolution Of The Field Of Ecological Restorati...
type                                                   journal-article
publication_year                                                  2018
issue                                                                3
volume                                                              27
chapter                                                               
publication_venue                                  Restoration Ecology
venue_type                                                     journal
publisher                                                 crossref:311
event                                                                 
Name: 281, dtype: object

The index of the current row is 282
The content of the row is as follows:
id                                   doi:10.1

The index of the current row is 469
The content of the row is as follows:
id                                              doi:10.3390/en14133917
title                Which Building Services Are Considered To Have...
type                                                   journal-article
publication_year                                                  2021
issue                                                               13
volume                                                              14
chapter                                                               
publication_venue                                             Energies
venue_type                                                     journal
publisher                                                crossref:1968
event                                                                 
Name: 469, dtype: object

The index of the current row is 470
The content of the row is as follows:
id                                           

In [6]:
for row_idx, row in df_publications.iterrows():
    print("\nRow index", row_idx)
    for item_idx, item in row.iteritems():
        print(item_idx, "-->", item)


Row index 0
id --> doi:10.1162/qss_a_00023
title --> Opencitations, An Infrastructure Organization For Open Scholarship
type --> journal-article
publication_year --> 2020
issue --> 1
volume --> 1
chapter --> 
publication_venue --> Quantitative Science Studies
venue_type --> journal
publisher --> crossref:281
event --> 

Row index 1
id --> doi:10.1007/s11192-019-03217-6
title --> Software Review: Coci, The Opencitations Index Of Crossref Open Doi-To-Doi Citations
type --> journal-article
publication_year --> 2019
issue --> 2
volume --> 121
chapter --> 
publication_venue --> Scientometrics
venue_type --> journal
publisher --> crossref:297
event --> 

Row index 2
id --> doi:10.1007/s11192-019-03311-9
title --> Nine Million Book Items And Eleven Million Citations: A Study Of Book-Based Scholarly Communication Using Opencitations
type --> journal-article
publication_year --> 2019
issue --> 2
volume --> 122
chapter --> 
publication_venue --> Scientometrics
venue_type --> journal
publisher -

issue --> 1
volume --> 115
chapter --> 
publication_venue --> Scientometrics
venue_type --> journal
publisher --> crossref:297
event --> 

Row index 54
id --> doi:10.1007/s11192-017-2636-z
title --> The Presence Of The Encyclicals In Web Of Science: A Bibliometric Approach
type --> journal-article
publication_year --> 2018
issue --> 1
volume --> 115
chapter --> 
publication_venue --> Scientometrics
venue_type --> journal
publisher --> crossref:297
event --> 

Row index 55
id --> doi:10.1007/s11192-018-2651-8
title --> Bibliometric Analysis To Identify An Emerging Research Area: Public Relations Intelligence—A Challenge To Strengthen Technological Observatories In The Network Society
type --> journal-article
publication_year --> 2018
issue --> 3
volume --> 115
chapter --> 
publication_venue --> Scientometrics
venue_type --> journal
publisher --> crossref:297
event --> 

Row index 56
id --> doi:10.1007/s11192-018-2705-y
title --> A Look Back Over The Past 40 Years Of Female Entrepreneurs

volume --> 11
chapter --> 
publication_venue --> Sustainability
venue_type --> journal
publisher --> crossref:1968
event --> 

Row index 105
id --> doi:10.3390/su11061677
title --> Adoption Of Systemic And Socio-Technical Perspectives In Waste Management, Weee And Elv Research
type --> journal-article
publication_year --> 2019
issue --> 6
volume --> 11
chapter --> 
publication_venue --> Sustainability
venue_type --> journal
publisher --> crossref:1968
event --> 

Row index 106
id --> doi:10.3390/su11092526
title --> A Bibliometric And Visualization Analysis Of Socially Responsible Funds
type --> journal-article
publication_year --> 2019
issue --> 9
volume --> 11
chapter --> 
publication_venue --> Sustainability
venue_type --> journal
publisher --> crossref:1968
event --> 

Row index 107
id --> doi:10.3390/su11113049
title --> The Role Of Smart Contracts In Sustainability: Worldwide Research Trends
type --> journal-article
publication_year --> 2019
issue --> 11
volume --> 11
chapter -->

event --> 

Row index 151
id --> doi:10.1016/j.jbusres.2018.12.002
title --> A Bibliometric Research In The Tourism, Leisure And Hospitality Fields
type --> journal-article
publication_year --> 2019
issue --> 
volume --> 101
chapter --> 
publication_venue --> Journal Of Business Research
venue_type --> journal
publisher --> crossref:78
event --> 

Row index 152
id --> doi:10.1016/j.jbusres.2019.02.050
title --> Ethics And Entrepreneurship: A Bibliometric Study And Literature Review
type --> journal-article
publication_year --> 2019
issue --> 
volume --> 99
chapter --> 
publication_venue --> Journal Of Business Research
venue_type --> journal
publisher --> crossref:78
event --> 

Row index 153
id --> doi:10.1016/j.jclepro.2020.122945
title --> Agricultural Co-Operatives In The Western World: A Bibliometric Analysis
type --> journal-article
publication_year --> 2020
issue --> 
volume --> 273
chapter --> 
publication_venue --> Journal Of Cleaner Production
venue_type --> journal
publisher

event --> 

Row index 182
id --> doi:10.1016/j.jclepro.2020.125751
title --> Micro-Electromechanical Systems-Based Technologies For Leak Detection And Localization In Water Supply Networks: A Bibliometric And Systematic Review
type --> journal-article
publication_year --> 2021
issue --> 
volume --> 289
chapter --> 
publication_venue --> Journal Of Cleaner Production
venue_type --> journal
publisher --> crossref:78
event --> 

Row index 183
id --> doi:10.1016/j.conbuildmat.2018.03.147
title --> A Holistic Review Of Cement Composites Reinforced With Graphene Oxide
type --> journal-article
publication_year --> 2018
issue --> 
volume --> 171
chapter --> 
publication_venue --> Construction And Building Materials
venue_type --> journal
publisher --> crossref:78
event --> 

Row index 184
id --> doi:10.1016/j.techfore.2020.120118
title --> Digital Academic Entrepreneurship: A Structured Literature Review And Avenue For A Research Agenda
type --> journal-article
publication_year --> 2020
issue 

...

event --> 

Row index 292
id --> doi:10.1007/s12078-018-9243-0
title --> Bibliometric Study On Functional Magnetic Resonance Imaging Literature (1995–2017) Concerning Chemosensory Perception
type --> journal-article
publication_year --> 2018
issue --> 1
volume --> 11
chapter --> 
publication_venue --> Chemosensory Perception
venue_type --> journal
publisher --> crossref:297
event --> 

Row index 293
id --> doi:10.1007/s12145-019-00408-w
title --> Ten Years Of Disaster Management And Use Of Ict: A Scientometric Analysis
type --> journal-article
publication_year --> 2019
issue --> 1
volume --> 13
chapter --> 
publication_venue --> Earth Science Informatics
venue_type --> journal
publisher --> crossref:297
event --> 

Row index 294
id --> doi:10.1007/978-3-319-91473-2_1
title --> A Bibliometric Analysis Of The Explainable Artificial Intelligence Research Field
type --> book-chapter
publication_year --> 2018
issue --> 
volume --> 
chapter --> 1
publication_venue --> Communications In Compu

id --> doi:10.1590/2318-08892018000300001
title --> Análisis De Co-Palabras Aplicado A Los Artículos Muy Citados En Biblioteconomía Y Ciencias De La Información (2007-2017)
type --> journal-article
publication_year --> 2018
issue --> 3
volume --> 30
chapter --> 
publication_venue --> Transinformação
venue_type --> journal
publisher --> crossref:530
event --> 

Row index 340
id --> doi:10.1590/2318-0889201931e190027
title --> Estudos Altmétricos No Brasil: Uma Análise A Partir Dos Currículos Da Plataforma Lattes-Cnpq
type --> journal-article
publication_year --> 2019
issue --> 
volume --> 31
chapter --> 
publication_venue --> Transinformação
venue_type --> journal
publisher --> crossref:530
event --> 

Row index 341
id --> doi:10.3897/rio.2.e9841
title --> Bibliometric Study To Assist Research Topic Selection: A Case From Research Design On Jakarta’S Groundwater (Part 1)
type --> journal-article
publication_year --> 2016
issue --> 
volume --> 2
chapter --> 
publication_venue --> Researc

title --> Internet Of Things (Iot) Technology Research In Business And Management Literature: Results From A Co-Citation Analysis
type --> journal-article
publication_year --> 2021
issue --> 6
volume --> 16
chapter --> 
publication_venue --> Journal Of Theoretical And Applied Electronic Commerce Research
venue_type --> journal
publisher --> crossref:1968
event --> 

Row index 388
id --> doi:10.1080/15440478.2021.1952139
title --> Bibliometric Analysis Of Cotton Research From Plant Sciences Category Based On Web Of Science
type --> journal-article
publication_year --> 2021
issue --> 
volume --> 
chapter --> 
publication_venue --> Journal Of Natural Fibers
venue_type --> journal
publisher --> crossref:301
event --> 

Row index 389
id --> doi:10.17533/udea.le.n95a344139
title --> Dinámicas De La Producción Científica Colombiana En Economía
type --> journal-article
publication_year --> 2021
issue --> 95
volume --> 
chapter --> 
publication_venue --> Lecturas De Economía
venue_type --> jour

Row index 439
id --> doi:10.3390/infrastructures6020021
title --> Dynamic Planning Of Construction Site For Linear Projects
type --> journal-article
publication_year --> 2021
issue --> 2
volume --> 6
chapter --> 
publication_venue --> Infrastructures
venue_type --> journal
publisher --> crossref:1968
event --> 

Row index 440
id --> doi:10.3846/jcem.2021.15260
title --> Application Of Multiple Criteria Decision Making Methods In Construction: A Systematic Literature Review
type --> journal-article
publication_year --> 2021
issue --> 6
volume --> 27
chapter --> 
publication_venue --> Journal Of Civil Engineering And Management
venue_type --> journal
publisher --> crossref:2209
event --> 

Row index 441
id --> doi:10.1177/15533506211026411
title --> A Bibliometric Analysis Of Overall And Top 100 Most-Cited Studies About Robotic Surgery Versus Open Surgery
type --> journal-article
publication_year --> 2021
issue --> 
volume --> 
chapter --> 
publication_venue --> Surgical Innovation
venue

chapter --> 
publication_venue --> Administrative Sciences
venue_type --> journal
publisher --> crossref:1968
event --> 

Row index 495
id --> doi:10.3390/admsci10030069
title --> Performance Analysis And Science Mapping Of Institutional Entrepreneurship Research
type --> journal-article
publication_year --> 2020
issue --> 3
volume --> 10
chapter --> 
publication_venue --> Administrative Sciences
venue_type --> journal
publisher --> crossref:1968
event --> 

Row index 496
id --> doi:10.1186/s12888-020-02825-4
title --> Mapping The Literature On Parents With Mental Illness, Across Psychiatric Sub-Disciplines: A Bibliometric Review
type --> journal-article
publication_year --> 2020
issue --> 1
volume --> 20
chapter --> 
publication_venue --> Bmc Psychiatry
venue_type --> journal
publisher --> crossref:297
event --> 

Row index 497
id --> doi:10.1080/00472778.2020.1776578
title --> Evolution Of The Entrepreneurship And Innovation Research In Ibero-America Between 1986 And 2015
type --> jo

In [7]:
for column_name, column in df_publications.items():
    print("\nThe name of the current column is", column_name)
    print("The content of the column is as follows:")
    print(column)


The name of the current column is id
The content of the column is as follows:
0                doi:10.1162/qss_a_00023
1         doi:10.1007/s11192-019-03217-6
2         doi:10.1007/s11192-019-03311-9
3              doi:10.1038/sdata.2016.18
4       doi:10.1371/journal.pbio.3000385
                     ...                
495           doi:10.3390/admsci10030069
496       doi:10.1186/s12888-020-02825-4
497    doi:10.1080/00472778.2020.1776578
498       doi:10.1007/s11301-020-00196-4
499    doi:10.1080/08989621.2020.1836620
Name: id, Length: 500, dtype: string

The name of the current column is title
The content of the column is as follows:
0      Opencitations, An Infrastructure Organization ...
1      Software Review: Coci, The Opencitations Index...
2      Nine Million Book Items And Eleven Million Cit...
3      The Fair Guiding Principles For Scientific Dat...
4      The Nih Open Citation Collection: A Public Acc...
                             ...                        
495    Pe

It seems that there are no values for chapters and events

In [9]:
df_publications.query("type == 'journal-article'")

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
0,doi:10.1162/qss_a_00023,"Opencitations, An Infrastructure Organization ...",journal-article,2020,1,1,,Quantitative Science Studies,journal,crossref:281,
1,doi:10.1007/s11192-019-03217-6,"Software Review: Coci, The Opencitations Index...",journal-article,2019,2,121,,Scientometrics,journal,crossref:297,
2,doi:10.1007/s11192-019-03311-9,Nine Million Book Items And Eleven Million Cit...,journal-article,2019,2,122,,Scientometrics,journal,crossref:297,
3,doi:10.1038/sdata.2016.18,The Fair Guiding Principles For Scientific Dat...,journal-article,2016,1,3,,Scientific Data,journal,crossref:297,
4,doi:10.1371/journal.pbio.3000385,The Nih Open Citation Collection: A Public Acc...,journal-article,2019,10,17,,Plos Biology,journal,crossref:340,
...,...,...,...,...,...,...,...,...,...,...,...
495,doi:10.3390/admsci10030069,Performance Analysis And Science Mapping Of In...,journal-article,2020,3,10,,Administrative Sciences,journal,crossref:1968,
496,doi:10.1186/s12888-020-02825-4,Mapping The Literature On Parents With Mental ...,journal-article,2020,1,20,,Bmc Psychiatry,journal,crossref:297,
497,doi:10.1080/00472778.2020.1776578,Evolution Of The Entrepreneurship And Innovati...,journal-article,2020,,,,Journal Of Small Business Management,journal,crossref:301,
498,doi:10.1007/s11301-020-00196-4,Intellectual Structure Of Management Innovatio...,journal-article,2020,3,71,,Management Review Quarterly,journal,crossref:297,


Almost all the publications have type journal article: 478
What types are the remaining 22??

In [10]:
df_publications.query("type == 'book-chapter'")

,id,title,type,publication_year,issue,volume,chapter,publication_venue,venue_type,publisher,event
8,doi:10.1007/978-3-030-61244-3_16,Researchflow: Understanding The Knowledge Flow...,book-chapter,2020,,,1,Lecture Notes In Computer Science - Knowledge ...,book,crossref:297,
9,doi:10.1007/978-3-030-61244-3_6,Ontologies Supporting Research-Related Informa...,book-chapter,2020,,,1,Lecture Notes In Computer Science - Knowledge ...,book,crossref:297,
10,doi:10.1007/978-3-030-54956-5_2,Question Answering On Scholarly Knowledge Graphs,book-chapter,2020,,,1,Digital Libraries For Open Knowledge - Lecture...,book,crossref:297,
11,doi:10.1007/978-3-030-55814-7_15,Dingo: An Ontology For Projects And Grants Lin...,book-chapter,2020,,,1,"Adbis, Tpdl And Eda 2020 Common Workshops And ...",book,crossref:297,
13,doi:10.1007/978-3-030-62466-8_28,The Opencitations Data Model,book-chapter,2020,,,1,Lecture Notes In Computer Science - The Semant...,book,crossref:297,
15,doi:10.1007/978-3-030-77385-4_37,Kgbench: A Collection Of Knowledge Graph Datas...,book-chapter,2021,,,1,The Semantic Web - Lecture Notes In Computer S...,book,crossref:297,
20,doi:10.1007/978-3-030-84825-5_11,Lobd: Linked Data Dashboard For Marine Biodive...,book-chapter,2021,,,1,Communications In Computer And Information Sci...,book,crossref:297,
26,doi:10.1007/978-3-030-16187-3_20,Using The Spar Ontology Network To Represent T...,book-chapter,2019,,,1,Advances In Intelligent Systems And Computing ...,book,crossref:297,
29,doi:10.1007/978-3-030-59194-6_37,Construction And Leverage Scientific Knowledge...,book-chapter,2020,,,1,Systems And Information Sciences - Advances In...,book,crossref:297,
30,doi:10.1007/978-3-030-61244-3_7,A Unified Nanopublication Model For Effective ...,book-chapter,2020,,,1,Lecture Notes In Computer Science - Knowledge ...,book,crossref:297,


In [14]:
import pandas as pd 
#loading the json file
from json import load

with open("import/relational_other_data.json", "r", encoding="utf-8") as f:
    json_doc = load(f)
    
print(type(json_doc))    

<class 'dict'>


In [15]:
from json import dump

with open("import/relational_other_data_mod.json", "w", encoding="utf-8") as f:
    dump(json_doc, f, ensure_ascii=False, indent=4)

In [27]:
import pandas as pd
import json
from sqlalchemy import create_engine

class RelationalProcessor:
    def __init__(self, db_path):
        self.db_path = db_path
        self.engine = create_engine(f"sqlite:///{db_path}") 

    def upload_data(self, file_path):
        """
        Uploads data from a CSV or JSON file to the database.

        Args:
            file_path (str): Path to the file to be uploaded.

        Returns:
            bool: True if the upload was successful, False otherwise.
        """
        try:
            if file_path.endswith('.csv'):
                df = pd.read_csv(file_path, dtype={'publication_year': 'Int64'}) 
                df['internal_id'] = df.index  # Assign unique internal IDs

            elif file_path.endswith('.json'):
                with open(file_path, 'r') as f:
                    data = json.load(f)
                df = pd.json_normalize(data) 

            else:
                raise ValueError("Unsupported file type. Please provide a CSV or JSON file.")

            # Create tables in database (if not exist)
            df.to_sql('publications', self.engine, if_exists='replace', index=False) 

            return True
        except Exception as e:
            print(f"Error uploading data: {e}")
            return False

# Example usage
db_path = 'my_database.db' 
processor = RelationalProcessor(db_path)

csv_file_path = 'import/relational_publications.csv'
json_file_path = 'import/relational_other_data.json'

if processor.upload_data(csv_file_path):
    print("CSV data uploaded successfully.")

if processor.upload_data(json_file_path):
    print("JSON data uploaded successfully.")

CSV data uploaded successfully.
Error uploading data: (sqlite3.InterfaceError) Error binding parameter 0 - probably unsupported type.
[SQL: INSERT INTO publications ("authors.doi:10.1162/qss_a_00023", "authors.doi:10.1007/s11192-019-03217-6", "authors.doi:10.1007/s11192-019-03311-9", "authors.doi:10.1038/sdata.2016.18", "authors.doi:10.1371/journal.pbio.3000385", "authors.doi:10.3233/ds-190016", "authors.doi:10.1007/s11192-020-03397-6", "authors.doi:10.1186/s13321-020-00448-1", "authors.doi:10.1007/978-3-030-61244-3_16", "authors.doi:10.1007/978-3-030-61244-3_6", "authors.doi:10.1007/978-3-030-54956-5_2", "authors.doi:10.1007/978-3-030-55814-7_15", "authors.doi:10.1007/s11192-020-03690-4", "authors.doi:10.1007/978-3-030-62466-8_28", "authors.doi:10.1038/s41597-020-00749-y", "authors.doi:10.1007/978-3-030-77385-4_37", "authors.doi:10.1007/s11192-021-04079-7", "authors.doi:10.1162/qss_a_00112", "authors.doi:10.7717/peerj-cs.421", "authors.doi:10.1101/2021.05.04.442638", "authors.doi:10.1

In [40]:
import pandas as pd
import json

def merge_csv_json(csv_file, json_file, csv_key, json_key):
    """
    Merges data from a CSV and JSON file into a single DataFrame.

    Args:
        csv_file: Path to the CSV file.
        json_file: Path to the JSON file.
        csv_key: Name of the key in the CSV file for merging.
        json_key: Name of the key in the JSON file for merging.

    Returns:
        pandas.DataFrame: A DataFrame containing the merged data.
    """

    try:
        # Load CSV data
        csv_df = pd.read_csv(csv_file)

        # Load JSON data
        with open(json_file, 'r') as f:
            json_data = json.load(f)

        # Convert JSON data to DataFrame 
        json_df = pd.DataFrame(json_data) 

        # Merge DataFrames based on the specified keys
        merged_df = pd.merge(csv_df, json_df, left_on=csv_key, right_on=json_key, how='inner') 

        return merged_df

    except FileNotFoundError:
        print(f"Error: File not found. Please check the paths for {csv_file} and {json_file}.")
        return None
    except Exception as e:
        print(f"An error occurred during merging: {e}")
        return None

# Example Usage
csv_path = 'import/relational_publications.csv'
json_path = 'import/relational_other_data.json'
csv_key = 'id'  # Replace with the actual key in your CSV
json_key = 'authors'  # Replace with the actual key in your JSON

merged_data = merge_csv_json(csv_path, json_path, csv_key, json_key)

if merged_data is not None:
    print(merged_data)

An error occurred during merging: unhashable type: 'list'
